In [1]:
# Dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests

In [39]:
traffic_raw = []
#Set up the api url and parameters
offset = 0
r = requests.get("https://data.austintexas.gov/resource/dx9v-zd7x.json", params = {
            "$limit": 50,
            "$offset": offset
            })
if r.status_code == 200:
    traffic_raw = r.json()
else:
    print("***** IT IS BROKEN *****")
    print(r.status_code)

In [40]:
pd.DataFrame(traffic_raw)
#Munge the json output that was converted to a data frame
traffic_raw = [{
    "ReportID": rec["traffic_report_id"],
    "PublishedDate": rec["published_date"],
    "IncidentDesc": rec["issue_reported"],
    "Address": rec["address"],
    "Latitude": rec["latitude"],
    "Longitude": rec["longitude"],
    "IncidentDate": rec["traffic_report_status_date_time"],
    "ReportStatus": rec["traffic_report_status"]
} for rec in traffic_raw]
#Convert back to data frame
traffic_incidents = pd.DataFrame(traffic_raw)

In [41]:
traffic_incidents

,ReportID,PublishedDate,IncidentDesc,Address,Latitude,Longitude,IncidentDate,ReportStatus
0,EBF5283D22437BB974F4248BA0EC4E90F26D32CB_15201...,2018-03-04T08:36:00.000Z,LOOSE LIVESTOCK,N Fm 973 Rd & Fm 969 Rd,30.256997,-97.611818,1970-01-18T14:16:00.000Z,ARCHIVED
1,9AE17B153A32567D1646CAA8C4010174C30AC349_15088...,2017-10-24T15:48:00.000Z,Crash Urgent,12700 Metric Blvd,30.415159,-97.688372,2017-10-24T15:51:00.000Z,ARCHIVED
2,A6990ED96E2C5ACAC92ACDCC3F83BA4E2893AD76_15088...,2017-10-24T15:49:00.000Z,Traffic Hazard,300 W Ben White Blvd Wb,30.224758,-97.766467,2017-10-24T15:51:00.000Z,ARCHIVED
3,551220DEEB362077F5DF356BDEBAF94F34F93F0C_15088...,2017-10-24T15:49:00.000Z,Crash Urgent,E 15th St & Red River St,30.275603,-97.734873,2017-10-24T15:51:00.000Z,ARCHIVED
4,0200320AE3BC247E747EF93E3259BC29DDF53392_15088...,2017-10-24T15:48:00.000Z,Crash Urgent,2414 Guadalupe St,30.288241,-97.741506,2017-10-24T15:51:00.000Z,ARCHIVED
5,7A7EA1512BC5014FD1B25FF389938BC75D25315D_15088...,2017-10-24T21:28:00.000Z,zSTALLED VEHICLE,4800 W Us 290 Hwy Wb,30.233755,-97.812873,2017-10-24T21:30:00.000Z,ARCHIVED
6,EAEBB5DB2B93D9B0E80FF737B2983A774CDED173_15088...,2017-10-24T21:30:00.000Z,COLLISN/ LVNG SCN,E Fm 1626 Rd & Old San Antonio Rd,30.142828,-97.801555,2017-10-24T21:33:00.000Z,ARCHIVED
7,1DD66758D96090B0762241E431C16CC14416C1F1_15088...,2017-10-24T21:41:00.000Z,COLLISION,11300 Old San Antonio Rd,30.14032,-97.803116,2017-10-24T21:42:00.000Z,ARCHIVED
8,F27CAC1218484BCB9D75A88232C544BFB19FA9A3_15088...,2017-10-24T22:14:00.000Z,zSTALLED VEHICLE,Fm 1825 Rd & W Wells Branch Pkwy,30.441168,-97.666175,2017-10-24T22:15:00.000Z,ARCHIVED
9,5F89685F47C1D42EFF25DD4ED40967B36B1A9C30_15088...,2017-10-24T19:36:00.000Z,Crash Service,6607 S Ih 35 Svrd Nb,30.190173,-97.770174,2017-10-24T22:15:00.000Z,ARCHIVED
